## Pre-trained model

In [6]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from transformers import Trainer, TrainingArguments
import pandas as pd

# Load pretrained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
#model = BertModel.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased")


# text = "Replace me by any text you'd like."
# encoded_input = tokenizer(text, return_tensors='pt')
# output = model(**encoded_input)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
import os 
os.environ["HF_HOME"] = "C:/Users/kodom/.cache/huggingface/datasets"

## Dataset for training

In [3]:
from datasets import load_dataset

dataset_name = "c4"
task_name = "en"
dataset = load_dataset(dataset_name, task_name)

shuffled_dataset = dataset['train'].shuffle(seed=42)

subset_dataset = shuffled_dataset.select([i for i in range(100)])


Generating train split:   1%|▏         | 4634999/364868892 [06:20<8:12:51, 12181.85 examples/s] 


PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:/Users/kodom/.cache/huggingface/datasets/c4/en/0.0.0/df532b158939272d032cc63ef19cd5b83e9b4d00c922b833e4cb18b2e9869b01.incomplete\\c4-train-00000-00020-of-NNNNN.arrow'

## INSTEAD OF IMPORTING DOWNLOADED A CSV

In [8]:
df = pd.read_csv("finetune.csv")

df['tokenized'] = df['text'].map(lambda x: tokenizer(x, truncation=True, padding='max_length', return_tensors='pt', max_length=512))

tensor_dataset = torch.utils.data.TensorDataset(*[torch.stack(df['tokenized'].apply(lambda x: x[key]).tolist()) for key in df['tokenized'].iloc[0]])

# Finetuning 

In [10]:
training_args = TrainingArguments(
    output_dir="./output",
    per_device_train_batch_size=4,
    num_train_epochs=3,
    save_steps=1000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tensor_dataset,
    data_collator=lambda data: data
)

trainer.train()


TypeError: BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,),

In [ ]:
trainer.evaluate()